# <연애 상담 챗봇(custom_data)(answer_object)>

### v5

### 기능:
#### + 랜덤 출력 기능
#### + 중복 답변 일시 제거
#### + 주기적 사용자 언급
#### + 예외 input  처리/수집

### 코드 수정:
#### global unexpected_input 변수명 -> global unexpected_input_list 로 통일
#### 응애! 아기 챗봇! cos_sim 수정
#### output_threshold 변수명 -> input_threshold로 수정
#### input_threshold = 0.6으로
#### 챗봇 답변 객체화, 변수명 : answer

## 1. 모델/데이터 불러오기 

In [2]:
!pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import random

# SBERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 학습된 input, output df 불러오기
data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v5(custom_data)(answer_object)/data/csv/연애시뮬레이션(실험7)_전처리.csv')
data = data[['input','output']]

# input별 embedding data 불러오기
embedding_data = torch.load('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v5(custom_data)(answer_object)/data/embedding_data/custom_love_embedding_data.pt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6a9bfcbd44f1808eb7ee0478a04bd81a61a52358d6126f99447ffbc1d2819e4f
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


## 2. 챗봇 대화 함수 선언

In [10]:
# 가장 최근 input
last_sentence = ''

# 가장 최근 출력 idx
last_sim_idx_list = []

# 현재 중복 cnt
overlap_cnt = 0

# 학습에 없는 질문 리스트
unexpected_input_list = []

def talk_to_bot(sentence):

  global last_sentence 
  global last_sim_idx_list
  global overlap_cnt
  global unexpected_input_list

  # 최종 출력 답변
  answer = ''
  



  # 알아듣는 문장 유사도 임계값
  input_thresholds = 0.6

  # 중복 문장 유사도 임계값
  overlap_thresholds = 0.85
  remove_sim_idx = None

  # 띄어쓰기 있는 사용자 input
  origin_sentence = sentence

  sentence = sentence.replace(" ","")
  sentence_encode = model.encode(sentence)
  sentence_tensor = torch.tensor(sentence_encode)

  last_sentence = last_sentence.replace(" ","")
  last_sentence_encode = model.encode(last_sentence)
  last_sentence_tensor = torch.tensor(last_sentence_encode)

  overlap_cos_sim = util.cos_sim(sentence_tensor, last_sentence_tensor)

  cos_sim = util.cos_sim(sentence_tensor, embedding_data)






  rand_num1 = random.random()
  # 예상 외의 input 입력 시
  if cos_sim[0][int(np.argmax(cos_sim))] < input_thresholds:

    answer = '응애 나 아기 챗봇!<br>문장 유사도 : '+ str(round(float( cos_sim[0][int(np.argmax(cos_sim))]),2))
    answer += '<br>새로운 데이터로 아기 챗봇 ' + str({round(rand_num1*50,1)}) + '% 성장!'

    print(answer)

    unexpected_input_list.append(origin_sentence)
    
    return last_sentence, last_sim_idx_list, overlap_cnt, answer
    

  # 이전 질문과 문장이 0.85이상 유사하면 
  # 해당 질문-답변 출력 X
  # 총 3번까지
  if overlap_cos_sim >= overlap_thresholds and overlap_cnt < 3 :
    print('************************ 이전 질문과 유사함 ************************')
    remove_sim_idx_list = last_sim_idx_list

    for i in remove_sim_idx_list:
      cos_sim[0][int(i)] = 0
    overlap_cnt += 1
    print(f'이전 출력 idx: {last_sim_idx_list}')
    print(f'이전 질문과 유사도: {overlap_cos_sim}')
    print(f'중복횟수 : {overlap_cnt}')
  else:
    overlap_cnt = 0
    last_sim_idx_list = []





  print(f'사용자의 질문: {sentence} \n')

  dot_score = util.dot_score(sentence_tensor, embedding_data)
  ascending_cos_sim = np.sort(cos_sim)
  descending_cos_sim = ascending_cos_sim[::-1]
  # print(f'cos_sim 유사도 내림차순: {ascending_cos_sim}')

  print(f'--------------------------- 최우선 순위 질문/답변 ---------------------------')
  print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")





  # 동일한 문장 유사도 input 일 때, 랜덤으로 출력

  max_cos_sim = cos_sim[0][np.argmax(cos_sim)]
  same_cos_sim_list = []

  for idx,one_cos_sim in enumerate(cos_sim[0]):
    if one_cos_sim == max_cos_sim:
      same_cos_sim_list.append(idx)

  print(f'same_cos_sim_list 크기: {len(same_cos_sim_list)}')
  if len(same_cos_sim_list) == 1:
    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최근 출력한 idx 기억
    last_sim_idx_list.append(int(np.argmax(cos_sim)))
  
  else:
    rand_num2 = random.choice(same_cos_sim_list)
    print(f'같은 문장 유사도 list:{same_cos_sim_list}')
    print(f'rand_num2 뽑음!')
    print(rand_num2)
    
    # data에서 선택된 질문 출력
    best_sim_idx = rand_num2
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최근 출력한 idx 기억
    last_sim_idx_list.append(int(best_sim_idx))







  # 중복 input 대응을 위해 이전sentence 저장 
  last_sentence = sentence

  # 최우선 순위 질문 벡터 초기화 
  cos_sim[0][best_sim_idx] = 0








  # 후순위 답변/질문 디버깅
  print(f'--------------------------- 후순위 답변/질문 디버깅 ---------------------------')
  for i in range(1,6):
    print(f'<{i}순위 Case>')
    print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최우선 순위 질문 벡터 초기화 
    cos_sim[0][best_sim_idx] = 0

  print(f'-------------------------------------------------------------------------------')

  return last_sentence, last_sim_idx_list , overlap_cnt , answer

## 3. 챗봇 실행 
## ex) talk_to_bot('ㅎㅇ')

In [11]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('연인이랑 무슨 얘기해?')

사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 234
same_cos_sim_list 크기: 1
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993596196174622
직접 수식 작성한 코사인 유사도 = 0.9482442736625671

답변 : 데이트 계획 같이 세워 봐~! 같이 하고싶은 게 많잖아

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 96
선택된 질문 = 헤어진 연인한테 연락왔는데 왜 연락한거지?
util.cos_sim 활용 코사인 유사도 : 0.7139381170272827
직접 수식 작성한 코사인 유사도 = 1.0012331008911133

답변 : 혹시 첫 멘트가 자니….? 야?

<2순위 Case>
가장 높은 코사인 유사도 idx : 231
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 할지 모르겠어.
util.cos_sim 활용 코사인 유사도 : 0.6888217926025391
직접 수식 작성한 코사인 유사도 = 1.0422461032867432

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<3순위 Case>
가장 높은 코사인 유사도 idx : 236
선택된 질문 = 여자친구랑 통화하는데 무슨 이야기 할 지 모르겠어
util.cos_sim 활용 코사인 유사도 : 0.6729137897491455
직접 수식 작성한 코사인 유사도 = 1.0346801280975342

답변 : 내가 이런 것 까지 가르쳐 줘야 하는거야~? ㅋㅋㅋㅋㅋ

<4순위 Case>
가장 높은 코사인 유사도 idx : 653
선택된 질문 = 헤어졌는데 전애인한테 연락하고 싶어. 어떻

In [12]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('어제 썸녀랑 밥먹었어')

응애 나 아기 챗봇!<br>문장 유사도 : 0.55<br>새로운 데이터로 아기 챗봇 {39.9}% 성장!


In [14]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('썸녀한테 고백해도 될까?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [438]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 1
사용자의 질문: 썸녀한테고백해도될까? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 492
same_cos_sim_list 크기: 1
선택된 질문 = 썸남한테 고백해도 될까?
util.cos_sim 활용 코사인 유사도 : 0.8112354278564453
직접 수식 작성한 코사인 유사도 = 0.9323492646217346

답변 : 먼저 그의 마음을 알아보기 위해 데이트 신청해보는 게 어때? 

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 446
선택된 질문 = 내일 썸녀한테 고백해도 될까?
util.cos_sim 활용 코사인 유사도 : 0.7682137489318848
직접 수식 작성한 코사인 유사도 = 0.9455898404121399

답변 : 좀 이르지 않아,,,?

<2순위 Case>
가장 높은 코사인 유사도 idx : 437
선택된 질문 = 오늘 썸녀한테 고백해도 될까?
util.cos_sim 활용 코사인 유사도 : 0.7678531408309937
직접 수식 작성한 코사인 유사도 = 0.9292535781860352

답변 : 좀 이르지 않아,,,?

<3순위 Case>
가장 높은 코사인 유사도 idx : 440
선택된 질문 = 썸녀에게 언제 고백할까?
util.cos_sim 활용 코사인 유사도 : 0.7576689124107361
직접 수식 작성한 코사인 유사도 = 0.9396165609359741

답변 : 먼저 그녀의 마음을 알아보기 위해 데이트 신청해보는 게 어때? 

<4순위 Cas

In [15]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('언제 모솔을 탈출할 수 있을 까요?')

응애 나 아기 챗봇!<br>문장 유사도 : 0.41<br>새로운 데이터로 아기 챗봇 {35.6}% 성장!


In [18]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('내일 짝녀 생일인데 선물 뭐 줄까?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [609, 608]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 2
사용자의 질문: 내일짝녀생일인데선물뭐줄까? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 610
same_cos_sim_list 크기: 1
선택된 질문 = 내일 생일인데 무슨 선물 해줄까?
util.cos_sim 활용 코사인 유사도 : 0.7088112831115723
직접 수식 작성한 코사인 유사도 = 0.9266941547393799

답변 : 생각보다 진심어린 편지를 좋아하는 사람들이 많더라고? 평소에 상대방이 갖고싶어했던 선물과 함께 편지를 주는 건 어때?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 613
선택된 질문 = 기념일이라서 선물 준비하려는데 뭐가 좋을까
util.cos_sim 활용 코사인 유사도 : 0.6418545246124268
직접 수식 작성한 코사인 유사도 = 0.8956409096717834

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 614
선택된 질문 = 여친한테 선물하려는데 뭐 하지?
util.cos_sim 활용 코사인 유사도 : 0.6258640289306641
직접 수식 작성한 코사인 유사도 = 0.9599776864051819

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데 같이 볼까? / [QR]띄우기

<3순위 Case>
가장 높은 코사인 유사도 idx : 612
선택된 질문 = 곧 남자친구 생일이더라고, 무슨 선물을 해줘야 기뻐할까?
util.cos_sim 활용 코사인 유사도 : 

In [25]:
last_sentence, last_sim_idx_list , overlap_cnt, answer = talk_to_bot('어제 남친이랑 헤어졌어')

사용자의 질문: 어제남친이랑헤어졌어 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 639
same_cos_sim_list 크기: 1
선택된 질문 = 어제 남친이랑 헤어졌어
util.cos_sim 활용 코사인 유사도 : 0.70563805103302
직접 수식 작성한 코사인 유사도 = 0.9115678071975708

답변 : 이별했구나? 너무 힘들겠다..

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 493
선택된 질문 = 오늘 썸남에게 고백하려고
util.cos_sim 활용 코사인 유사도 : 0.5632259845733643
직접 수식 작성한 코사인 유사도 = 0.9165316820144653

답변 : 상대방이 너무 갑작스럽게 느낄 수 있지 않을까?

<2순위 Case>
가장 높은 코사인 유사도 idx : 642
선택된 질문 = 여친이랑 헤어졌어
util.cos_sim 활용 코사인 유사도 : 0.559671938419342
직접 수식 작성한 코사인 유사도 = 0.949023425579071

답변 : 그동안 마음 고생 많았지? 지금은 좀 괜찮아?

<3순위 Case>
가장 높은 코사인 유사도 idx : 641
선택된 질문 = 나 헤어짐
util.cos_sim 활용 코사인 유사도 : 0.5464581847190857
직접 수식 작성한 코사인 유사도 = 0.8860830664634705

답변 : 무슨 일로 헤어지게 된 거야? 그동안 마음 고생이 많았겠다.

<4순위 Case>
가장 높은 코사인 유사도 idx : 439
선택된 질문 = 오늘 썸녀에게 고백하려고
util.cos_sim 활용 코사인 유사도 : 0.5461305975914001
직접 수식 작성한 코사인 유사도 = 0.9237662553787231



In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('관심있는 사람이 밀당 쩌는데 어떡하죠')

응애 나 아기 챗봇!
문장 유사도 : 0.46

새로운 데이터로 아기 챗봇 (35.9%) 성장!


In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('애인이랑 싸우고 연락을 안하고 있어요. 먼저 보내봐야 할까요?')

사용자의 질문: 애인이랑싸우고연락을안하고있어요.먼저보내봐야할까요? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 518
same_cos_sim_list 크기: 2
같은 문장 유사도 list:[518, 525]
rand_num2 뽑음!
518
선택된 질문 = 애인이랑 아직 만나면 어색해요. 어떻게하면 친해질 수 있을까요?
util.cos_sim 활용 코사인 유사도 : 0.672960638999939
직접 수식 작성한 코사인 유사도 = 0.9349518418312073

답변 : 같이 할 수 있는 취미 생활을 찾아보는 건 어때??

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 525
선택된 질문 = 애인이랑 아직 만나면 어색해요. 어떻게하면 친해질 수 있을까요?
util.cos_sim 활용 코사인 유사도 : 0.672960638999939
직접 수식 작성한 코사인 유사도 = 0.9349518418312073

답변 : 아직 상대에 대해 잘 모르기 때문에 어색하다고 느낄 수 있어. 그런데 이렇게 생각해 봐. 연애 초반이기 때문에 가질 수 있는 감정 아닐까? 그 사람과 친해질수록 어색함이란 감정은 옅어질거야. 이 순간의 어색한 두근거림을 즐겨 봐

<2순위 Case>
가장 높은 코사인 유사도 idx : 291
선택된 질문 = 연인과 싸웠는데 연락이 안 돼.. 헤어지기 싫은데, 먼저 연락 해야할까?
util.cos_sim 활용 코사인 유사도 : 0.6641734838485718
직접 수식 작성한 코사인 유사도 = 0.9797177910804749

답변 : 왜 아직 연락 안 한거야? 자존심 챙길 때가 아니라구!

<3순위 Case>
가장 높은 코사인 유사도 idx : 521
선택된 질문 = 애인이랑 아직 만나면 어색해요.

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('남친의 절친한테 고백받았는데 어떡하지?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [455]
이전 질문과 유사도: tensor([[0.9412]])
중복횟수 : 1
사용자의 질문: 남친의절친한테고백받았는데어떡하지? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 454
same_cos_sim_list 크기: 1
선택된 질문 = 남친의 절친이 나한테 고백했는데 어떡하지..?
util.cos_sim 활용 코사인 유사도 : 0.6868040561676025
직접 수식 작성한 코사인 유사도 = 0.9442533254623413

답변 : 친구 맞니?? 제정신이라면 못 그럴 거 같은데?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 458
선택된 질문 = 남친의 절친이 고백했어
util.cos_sim 활용 코사인 유사도 : 0.6098761558532715
직접 수식 작성한 코사인 유사도 = 0.8766645789146423

답변 : 친구 맞니?? 제정신이라면 못 그럴 거 같은데?

<2순위 Case>
가장 높은 코사인 유사도 idx : 96
선택된 질문 = 헤어진 연인한테 연락왔는데 왜 연락한거지?
util.cos_sim 활용 코사인 유사도 : 0.6096512079238892
직접 수식 작성한 코사인 유사도 = 0.8851876258850098

답변 : 혹시 첫 멘트가 자니….? 야?

<3순위 Case>
가장 높은 코사인 유사도 idx : 452
선택된 질문 = 남자친구의 절친이 나한테 고백했는데 어떡하지..?
util.cos_sim 활용 코사인 유사도 : 0.6077165603637695
직접 수식 작성한 코사인 유사도 = 0.9377195239067078

답변 :

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('안친한데 영화보자고 하면 부담스러우려나?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [533]
이전 질문과 유사도: tensor([[0.9663]])
중복횟수 : 1
사용자의 질문: 안친한데영화보자고하면부담스러우려나? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 532
same_cos_sim_list 크기: 1
선택된 질문 = 안 친한 사람이 영화보자고 하면 부담스럽겠죠?
util.cos_sim 활용 코사인 유사도 : 0.8030471801757812
직접 수식 작성한 코사인 유사도 = 0.9819521307945251

답변 : 안 친한 관계에서 영화를 보자고 하면 상대는 부담스러워서 너와 친해지기 더 힘들어 할 것 같아. 모든 일에는 순서가 있듯이 친해지고 싶어서 급하게 행동하지 말고, 그 사람과 친분을 조금 더 쌓고 난 뒤에 얘기해보는게 어떨까??

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 534
선택된 질문 = 관심가는 사람이 있는데 아직 안 친해서 어색해.. 갑자기 영화 보자고 하면 부담스러워 하려나?
util.cos_sim 활용 코사인 유사도 : 0.7555015683174133
직접 수식 작성한 코사인 유사도 = 1.005321741104126

답변 : 안 친한 관계에서 영화를 보자고 하면 상대는 부담스러워서 너와 친해지기 더 힘들어 할 것 같아. 모든 일에는 순서가 있듯이 친해지고 싶어서 급하게 행동하지 말고, 그 사람과 친분을 조금 더 쌓고 난 뒤에 얘기해보는게 어떨까??

<2순위 Case>
가장 높은 코사인 유사도 idx : 369
선택된 질문 = 남친을 좋아하는데 애정표현하기 너무 어색해
util.cos_sim 활용 코사인 유사도 : 0.5

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 5
같은 문장 유사도 list:[519, 520, 521, 522, 523]
rand_num2 뽑음!
522
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 요즘엔 쓸모없는 선물 주는 것도 유행이라던데? 어때? ㅎㅎㅎ

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<3순위 Case>
가장 높은 코사인 유사도 idx : 521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

<4순위 Case>
가장 높은 코사인 

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [522, 523]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 2
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 3
같은 문장 유사도 list:[519, 520, 521]
rand_num2 뽑음!
519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<2순위 Case>
가장 높은 코사인 유사도 idx : 521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

<3순위 Case>
가장 높은 코사인 유사도 idx : 529
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.70825

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [522, 523, 519]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 3
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 520
same_cos_sim_list 크기: 2
같은 문장 유사도 list:[520, 521]
rand_num2 뽑음!
521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276957511902
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<2순위 Case>
가장 높은 코사인 유사도 idx : 529
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7082570195198059
직접 수식 작성한 코사인 유사도 = 0.9903245568275452

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<3순위 Case>
가장 높은 코사인 유사도 idx : 530
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.70825

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [523, 522, 521]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 3
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 2
같은 문장 유사도 list:[519, 520]
rand_num2 뽑음!
519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<2순위 Case>
가장 높은 코사인 유사도 idx : 529
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7082570791244507
직접 수식 작성한 코사인 유사도 = 0.9903244972229004

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<3순위 Case>
가장 높은 코사인 유사도 idx : 530
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7082570791244507


In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('시도 때도 없이 연락해서 힘들어')

응애 나 아기 챗봇!
문장 유사도 : 0.49

새로운 데이터로 아기 챗봇 (38.9%) 성장!


In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('수료 이후에 취업할 수 있을거 같아?')

응애 나 아기 챗봇!
문장 유사도 : 0.39

새로운 데이터로 아기 챗봇 (22.2%) 성장!


In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구가 연락을 잘 안받아서 힘들어')

사용자의 질문: 여자친구가연락을잘안받아서힘들어 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 409
same_cos_sim_list 크기: 5
같은 문장 유사도 list:[409, 410, 411, 412, 413]
rand_num2 뽑음!
412
선택된 질문 = 여자친구가 요즘 화를 많이 내
util.cos_sim 활용 코사인 유사도 : 0.6662753224372864
직접 수식 작성한 코사인 유사도 = 1.0103453397750854

답변 : 이유없는 짜증과 화는 헤어짐의 신호로 볼 수 있어.

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 409
선택된 질문 = 여자친구가 요즘 화를 많이 내
util.cos_sim 활용 코사인 유사도 : 0.6662753224372864
직접 수식 작성한 코사인 유사도 = 1.0103453397750854

답변 : 그 사람한테 요즘 다른 힘든 일이 생긴 건 아니야?

<2순위 Case>
가장 높은 코사인 유사도 idx : 410
선택된 질문 = 여자친구가 요즘 화를 많이 내
util.cos_sim 활용 코사인 유사도 : 0.6662753224372864
직접 수식 작성한 코사인 유사도 = 1.0103453397750854

답변 : 연인에 대한 마음이 식었을 때 보이는 행동이긴 해..

<3순위 Case>
가장 높은 코사인 유사도 idx : 411
선택된 질문 = 여자친구가 요즘 화를 많이 내
util.cos_sim 활용 코사인 유사도 : 0.6662753224372864
직접 수식 작성한 코사인 유사도 = 1.0103453397750854

답변 : 상대에게 크게 문제가 있지 않았는데 헤어지고 싶을 때, 미안함때문에 되려 화내기도 한대. 그런데 그런 사람한테 잘해주는

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('어제 여자친구랑 싸웠어')

사용자의 질문: 어제여자친구랑싸웠어 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 37
same_cos_sim_list 크기: 5
같은 문장 유사도 list:[37, 38, 39, 40, 41]
rand_num2 뽑음!
38
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.546175479888916
직접 수식 작성한 코사인 유사도 = 1.0445338487625122

답변 : 와.. 너무 스트레스 받는데..? 연인이랑 진지하게 얘기해봤어?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 37
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.546175479888916
직접 수식 작성한 코사인 유사도 = 1.0445338487625122

답변 : 와.. 너무 스트레스 받는데..? 너는 괜찮아?

<2순위 Case>
가장 높은 코사인 유사도 idx : 39
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.546175479888916
직접 수식 작성한 코사인 유사도 = 1.0445338487625122

답변 : 헤어졌는데 친구로 지내는 게 가능한 거야?

<3순위 Case>
가장 높은 코사인 유사도 idx : 40
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.546175479888916
직접 수식 작성한 코사인 유사도 = 1.0445338487625122

답변 : 찜찜하게 엮인 관계여서 네가 상처 많이 받을 것 같아..

<4

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('어제 애인이랑 싸웠어')

응애 나 아기 챗봇!
문장 유사도 : 0.44

새로운 데이터로 아기 챗봇 (9.1%) 성장!


In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('오늘 짝녀한테 고백해도 좋을까?')

사용자의 질문: 오늘짝녀한테고백해도좋을까? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 82
same_cos_sim_list 크기: 6
같은 문장 유사도 list:[82, 83, 84, 85, 86, 87]
rand_num2 뽑음!
87
선택된 질문 = 어떻게 고백해야 할까?
util.cos_sim 활용 코사인 유사도 : 0.46506762504577637
직접 수식 작성한 코사인 유사도 = 0.9221333265304565

답변 : 무조건 단톡방에서 고백ㄱ! 모두가 축복해주는 연인이 되는거지~

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 82
선택된 질문 = 어떻게 고백해야 할까?
util.cos_sim 활용 코사인 유사도 : 0.46506762504577637
직접 수식 작성한 코사인 유사도 = 0.9221333265304565

답변 : 고백은 타이밍인거 알지? 서둘러 고백하려 하지 말고, 타이밍을 보고 고백해!

<2순위 Case>
가장 높은 코사인 유사도 idx : 83
선택된 질문 = 어떻게 고백해야 할까?
util.cos_sim 활용 코사인 유사도 : 0.46506762504577637
직접 수식 작성한 코사인 유사도 = 0.9221333265304565

답변 : 사람이 많은 곳에서 촛불로 하트를 만들고, 꽃다발을 준비해. 그 사람의 이름을 넣어 개사한 사랑의 세레나데를 불러 주며 꽃다발을 뙇! 게임 끝이지~ ㅋㅋㅋ

<3순위 Case>
가장 높은 코사인 유사도 idx : 84
선택된 질문 = 어떻게 고백해야 할까?
util.cos_sim 활용 코사인 유사도 : 0.46506762504577637
직접 수식 작성한 코사인 유사도 = 0.9221333265304565

답변 : 그 사람이 나랑 

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('내일 점심 데이트로 어떤 거 먹을까?')

사용자의 질문: 내일점심데이트로어떤거먹을까?" 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 508
same_cos_sim_list 크기: 6
같은 문장 유사도 list:[508, 509, 510, 511, 512, 513]
rand_num2 뽑음!
511
선택된 질문 = 선물 뭘 준비하는 게 좋을까?
util.cos_sim 활용 코사인 유사도 : 0.45368051528930664
직접 수식 작성한 코사인 유사도 = 0.9055710434913635

답변 : 요즘엔 쓸모없는 선물 주는 것도 유행이라던데? 어때? ㅎㅎㅎ

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 508
선택된 질문 = 선물 뭘 준비하는 게 좋을까?
util.cos_sim 활용 코사인 유사도 : 0.45368051528930664
직접 수식 작성한 코사인 유사도 = 0.9055710434913635

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 509
선택된 질문 = 선물 뭘 준비하는 게 좋을까?
util.cos_sim 활용 코사인 유사도 : 0.45368051528930664
직접 수식 작성한 코사인 유사도 = 0.9055710434913635

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<3순위 Case>
가장 높은 코사인 유사도 idx : 510
선택된 질문 = 선물 뭘 준비하는 게 좋을까?
util.cos_sim 활용 코사인 유사도 : 0.45368051528930664
직접 수식 작성한 코사인 유사도 = 0.9055710434913635

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

<4순위 Case>
가장 높은 코사인

In [ ]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('남자친구랑 일주일에 몇번 정도 만나는게 적당할까?')

사용자의 질문: 남자친구랑일주일에몇번정도만나는게적당할까? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 37
same_cos_sim_list 크기: 5
같은 문장 유사도 list:[37, 38, 39, 40, 41]
rand_num2 뽑음!
38
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.4746919572353363
직접 수식 작성한 코사인 유사도 = 0.8353371620178223

답변 : 와.. 너무 스트레스 받는데..? 연인이랑 진지하게 얘기해봤어?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 37
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.4746919572353363
직접 수식 작성한 코사인 유사도 = 0.8353371620178223

답변 : 와.. 너무 스트레스 받는데..? 너는 괜찮아?

<2순위 Case>
가장 높은 코사인 유사도 idx : 39
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.4746919572353363
직접 수식 작성한 코사인 유사도 = 0.8353371620178223

답변 : 헤어졌는데 친구로 지내는 게 가능한 거야?

<3순위 Case>
가장 높은 코사인 유사도 idx : 40
선택된 질문 = 남자친구가 전여자친구하고 계속 친구로 지내고 있더라.
util.cos_sim 활용 코사인 유사도 : 0.4746919572353363
직접 수식 작성한 코사인 유사도 = 0.8353371620178223

답변 : 찜찜하게 엮인 관계여서 네가 상처 